In [26]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'gproject.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import re
import json
from bs4 import BeautifulSoup
import string
# Output directory
import spacy
from pathlib import Path
from spacy.training import Example
from spacy import displacy
from IPython.core.display import display, HTML
from IPython.display import clear_output
import requests

In [8]:
from articles.models import Article, Pathway
from django.db.models import Exists

In [87]:
has_pathway = Article.objects.all().exclude(annotations=None).exclude(annotations=[])
[entry for entry in has_pathway]

[<Article: Article object (465146)>,
 <Article: Article object (465145)>,
 <Article: Article object (465144)>,
 <Article: Article object (465143)>,
 <Article: Article object (465142)>,
 <Article: Article object (465141)>,
 <Article: Article object (465140)>,
 <Article: Article object (465139)>,
 <Article: Article object (465138)>,
 <Article: Article object (465137)>,
 <Article: Article object (465136)>,
 <Article: Article object (465135)>,
 <Article: Article object (465134)>,
 <Article: Article object (465132)>,
 <Article: Article object (465131)>,
 <Article: Article object (465130)>,
 <Article: Article object (465129)>,
 <Article: Article object (465128)>,
 <Article: Article object (465127)>,
 <Article: Article object (465125)>,
 <Article: Article object (465124)>,
 <Article: Article object (465123)>,
 <Article: Article object (465122)>,
 <Article: Article object (465121)>,
 <Article: Article object (465120)>,
 <Article: Article object (465119)>,
 <Article: Article object (465118)>,
 

In [88]:
len(has_pathway)

447

In [52]:
TRAIN_DATA = []

i = 0
table = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}
for ar in has_pathway:
    abstractText = ar.abstract_text.lower()
    points = []
    for annot in ar.annotations:
        tokens = annot['tokens']
        start = annot['start']
        end = annot['end']
        words = []
        for tok in tokens:
            tok = tok.lower()
            soup = BeautifulSoup(tok, "html.parser")
            for elm in soup.find_all():
                elm.decompose()
            tok = soup.text
            tok = tok.translate(table)
            words.append(tok)
        pathway = " ".join(words)
        for match in re.finditer(pathway, abstractText):
            print(match)
            shouldPass = False
            if (match.start(),match.end(), "PATHWAY") not in points:
                for point in points:
                    if match.start() >= point[0] and match.end() <= point[1]:
                        shouldPass = True
                        break
                    if match.end() == point[1] and match.start() < point[0]:
                        point = (match.start(),match.end(), "PATHWAY")
                        shouldPass = True
                        break
                    if match.start() == point[0] and match.end() > point[1]:
                        point = (match.start(),match.end(), "PATHWAY")
                        shouldPass = True
                        break
                if not shouldPass:
                    points.append((match.start(),match.end(), "PATHWAY"))
        print(tokens, start, end)
    print(points)
    TRAIN_DATA.append((abstractText, {"entities": points}))
    print(i)
    print('------\n')
    i += 1

<re.Match object; span=(19, 29), match='urea cycle'>
['<h4>Background</h4>Urea', 'cycle'] 0 2
[(19, 29, 'PATHWAY')]
0
------

<re.Match object; span=(82, 92), match='urea cycle'>
['urea', 'cycle'] 11 13
[(82, 92, 'PATHWAY')]
1
------

<re.Match object; span=(270, 280), match='urea cycle'>
['urea', 'cycle'] 34 36
[(270, 280, 'PATHWAY')]
2
------

<re.Match object; span=(4, 14), match='urea cycle'>
['urea', 'cycle'] 1 3
[(4, 14, 'PATHWAY')]
3
------

<re.Match object; span=(647, 657), match='urea cycle'>
['urea', 'cycle'] 85 87
[(647, 657, 'PATHWAY')]
4
------

<re.Match object; span=(0, 10), match='urea cycle'>
['Urea', 'cycle'] 0 2
<re.Match object; span=(67, 98), match='nitrogen detoxification pathway'>
['nitrogen', 'detoxification', 'pathway'] 11 14
<re.Match object; span=(1313, 1337), match='tricarboxylic acid cycle'>
['tricarboxylic', 'acid', 'cycle,'] 184 187
[(0, 10, 'PATHWAY'), (67, 98, 'PATHWAY'), (1313, 1337, 'PATHWAY')]
5
------

<re.Match object; span=(287, 297), match='urea

In [84]:
import json

with open('annotated_data.json', 'w') as json_file:
    json.dump(TRAIN_DATA, json_file)

In [54]:
from sklearn.model_selection import ShuffleSplit
import numpy as np
rs = ShuffleSplit(n_splits=1, test_size=.25, random_state=0)

train = []
test = []
test_indexes = []

for train_index, test_index in rs.split(TRAIN_DATA):
    print(test_index)
    test_indexes = test_index
    train = np.array(TRAIN_DATA)[train_index.astype(int)]
    test = np.array(TRAIN_DATA)[test_index.astype(int)]
    
examples = []
for text, annots in train:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))
nlp.initialize(lambda: examples)

[357 416 302 410 141 363  12 343 362  90 377  54 284 240 423  21 122   1
  10 253 233  37 175 347 293 102 213 219 159 245 442 118  76 365 210 317
 446 403 135 348 306 194  96 301 375  78  60 393 443 113 239 170   6  71
 198 374  15 406 354 330 142 107  56 417 264 352 387 246 164 274 420 391
 241  59 351 250 432  65 134 332 124 153 144 331 229 200 440 155 329  49
  52  74  26  45 272 158   4 247 231 100 268   8 196   5 188 206 218   7
 312 394  22  68]


In [83]:
import json

with open('annotated_data.json') as f:
    data = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [43]:
output_dir=Path('/Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content3')

# Loading the model from the directory
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

Loading from /Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content3


In [153]:
ner = nlp.get_pipe('ner')

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [154]:
# Importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

    sizes = compounding(1.0, 4.0, 1.001)
    # Training for 30 iterations     
    for itn in range(30):
    # shuffle examples before training
        random.shuffle(examples)
        # batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=sizes)
        losses = {}
        for batch in batches:
            # Calling update() over the iteration
            nlp.update(batch, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 0.0}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.10235470533371}
Losses {'ner': 121.102354

In [80]:
test_examples = []
for text, annots in test:
    test_examples.append(Example.from_dict(nlp.make_doc(text), annots))

print(len(test_examples))
print(len(has_pathway))

scores = nlp.evaluate(test_examples)

112
447


In [74]:
print(scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.9387096774193548, 'ents_r': 0.9509803921568627, 'ents_f': 0.9448051948051949, 'ents_per_type': {'PATHWAY': {'p': 0.9387096774193548, 'r': 0.9509803921568627, 'f': 0.9448051948051949}}, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'speed': 40796.99714431319}


In [82]:
import pandas as pd

pd_data = (pd.DataFrame.from_dict(scores['ents_per_type'], orient='index'))
pd_data

,p,r,f
PATHWAY,0.93871,0.95098,0.944805


In [161]:
# Output directory
from pathlib import Path
output_dir=Path('/Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content3')
# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'pathway_ner_model'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /Users/omerfaruk.davarci/Documents/MetabolysmAnalysisAPI/content3
